In [ ]:
# !pip install langchain

In [ ]:
import os
import openai
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI


from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
openai.api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)


chat = ChatOpenAI(model='gpt-3.5-turbo')

In [ ]:
messages = [
    SystemMessage(content="You only answer in german."),
    HumanMessage(content="Tell me a joke."),
]

chat(messages)

In [ ]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template="""Interprete the text and evaluate the text.
sentiment: is the text positive, negative or neutral?
subject: What is the subject of the text? Use exactly one word not two or more.

Format the output as JSON with the following keys:
sentiment
subject
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
# using the ChatPromptTemplate to combine the two prompts
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat(chat_prompt.format_prompt(text="I ordered a kitchen table, but it was scratched").to_messages())

In [ ]:
# using a chain to run the prompt
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(text="I ordered a kitchen table, but it was scratched")

# Real World example with ResponseSchema, Templates, Chains and OutputParsers



In [ ]:
from langchain.output_parsers import (
    ResponseSchema,
    StructuredOutputParser
)

sentiment_schema = ResponseSchema(name="sentiment", description="Is the text positive, negative or neutral? Only use one of the provided words.")
subject_schema = ResponseSchema(name="subject", description="What is the subject of the text? Use exactly one word.")
price_schema = ResponseSchema(name="price", description="What is the price of the product? Use none if no price was provided in the text.")

response_schemas = [sentiment_schema, subject_schema, price_schema]

In [ ]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
format_instructions

In [ ]:
template="""Interprete the text and evaluate the text.
sentiment: is the text positive, negative or neutral?
subject: What is the subject of the text? Use exactly one word not two or more.

text: {input}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template)
format_instructions = parser.get_format_instructions()

messages = prompt.format_messages(input="I ordered a kitchen table, but it was scratched", format_instructions=format_instructions)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
response

In [ ]:
output_dict = parser.parse(response.content)
output_dict

# Using Sequential Chains to Pass Output from One Model to Another

In [ ]:
template="""Interprete the text and evaluate the text.
sentiment: is the text positive, negative or neutral?
subject: What is the subject of the text? Use exactly one word not two or more.

Format the output as JSON with the following keys:
sentiment
subject
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chain = LLMChain(llm=chat, prompt=chat_prompt)

response_template="""
You are a helpful bot that creates a "Thank you" message for a customer.
If the customer is unsatisfied, offer them a real world assistant to talk to.
You will get a sentiment and subject as input.

text: {input}
"""

review_template=PromptTemplate(input_variables=["input"], template=response_template)
review_chain = LLMChain(llm=chat, prompt=review_template)





In [ ]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[chain, review_chain], verbose=True)

overall_chain.run(input="I ordered a kitchen table, but it was scratched")